In [4]:
import	random
from	numpy.random	import	randint
from	numpy.random	import	rand
import	pandas	as	pd
import	datetime
import	numpy	as	np
import	matplotlib.pyplot	as	plt
import	mip
import	pycaret
from	pycaret.regression	import	*

In [12]:
df_0	=	pd.read_csv('data/공정 데이터 2022년 8월.csv')
df_0	=	df_0.drop(['idx',	'lineno',	'Order_date',	'cnt',	'day_night_type'],	axis=1)
df_1	=	pd.read_csv('data/로봇 1호-전류 데이터.csv',	index_col=0)
df_2	=	pd.read_csv('data/로봇 2호-전류 데이터.csv',	index_col=0)
df_3	=	pd.read_csv('data/프레스 1호-유압모터 전류데이터.csv',	index_col=0)
df_4	=	pd.read_csv('data/프레스 2호-유압모터 전류데이터.csv',	index_col=0)
df_5	=	pd.read_csv('data/프레스 3호-유압모터 전류데이터.csv',	index_col=0)
df_6	=	pd.read_csv('data/프레스 4호-유압모터 전류데이터.csv',	index_col=0)

In [14]:
 #	컬럼	이름	수정
df_0	=	df_0.rename(columns={'product_date':	'times',	'quantity':'qty'})
df_1	=	df_1.rename(columns={'Time_s[s]':	'times',	'RMS[A]':'amp_1'})
df_2	=	df_2.rename(columns={'Time_s[s]':	'times',	'Acceleration[g]':'amp_2'})
df_3	=	df_3.rename(columns={'Time_s[s]':	'times',	'RMS[A]':'amp_3'})
df_4	=	df_4.rename(columns={'Time_s[s]':	'times',	'RMS[A]':'amp_4'})
df_5	=	df_5.rename(columns={'Time_s[s]':	'times',	'RMS[A]':'amp_5'})
df_6	=	df_6.rename(columns={'Time_s[s]':	'times',	'RMS[A]':'amp_6'})
df_0['times']	=	pd.to_datetime(df_0['times'])
df_1['times']	=	pd.to_datetime(df_1['times'])
df_2['times']	=	pd.to_datetime(df_2['times'])
df_3['times']	=	pd.to_datetime(df_3['times'])
df_4['times']	=	pd.to_datetime(df_4['times'])
df_5['times']	=	pd.to_datetime(df_5['times'])
df_6['times']	=	pd.to_datetime(df_6['times'])

In [16]:
perc	=30
df_set	=	{'df_0':df_0, 'df_1':df_1, 'df_2':df_2, 'df_3':df_3, 'df_4':df_4, 'df_5':df_5, 'df_6':df_6}
for	df_name	in	list(df_set):
    print(f'DataFrame	name:	{df_name}')
    df	=	df_set[df_name]
    print('[step	1-1]')
    print(round(df.isnull().sum()/len(df)*100,	2))
    print('[step	1-2]')
    print(df.isnull().sum()/len(df)*100>perc)
    print('[step	2-1]')
    print(df.isnull().head())
    print('[step	2-2]')
    print(df.isnull().sum())
    print('[step	2-3]')
    cmpt_len	=	df.isnull().sum().sum()
    print(cmpt_len)
completeness	=	(1-cmpt_len/len(df))*100
print("결측치	=	%d개	\n완전성	품질지수	:	%.2f%%	"%(cmpt_len,completeness))

DataFrame	name:	df_0
[step	1-1]
itemno    0.0
times     0.0
qty       0.0
dtype: float64
[step	1-2]
itemno    False
times     False
qty       False
dtype: bool
[step	2-1]
   itemno  times    qty
0   False  False  False
1   False  False  False
2   False  False  False
3   False  False  False
4   False  False  False
[step	2-2]
itemno    0
times     0
qty       0
dtype: int64
[step	2-3]
0
DataFrame	name:	df_1
[step	1-1]
times    0.0
amp_1    0.0
dtype: float64
[step	1-2]
times    False
amp_1    False
dtype: bool
[step	2-1]
   times  amp_1
1  False  False
2  False  False
3  False  False
4  False  False
5  False  False
[step	2-2]
times    0
amp_1    0
dtype: int64
[step	2-3]
0
DataFrame	name:	df_2
[step	1-1]
times     0.0
RMS[A]    0.0
dtype: float64
[step	1-2]
times     False
RMS[A]    False
dtype: bool
[step	2-1]
   times  RMS[A]
1  False   False
2  False   False
3  False   False
4  False   False
5  False   False
[step	2-2]
times     0
RMS[A]    0
dtype: int64
[step	2-3]
0
DataFrame	name:	

In [19]:
unique_score	=0
df_set	=	{'df_0':df_0, 'df_1':df_1, 'df_2':df_2, 'df_3':df_3, 'df_4':df_4, 'df_5':df_5, 'df_6':df_6}
for df_name in	list(df_set):
    df	=df_set[df_name]
    check_unique = df.value_counts().reset_index()
    perc_check_unique = round((len(check_unique)-len(check_unique[check_unique[0]>1]))/len(check_unique)*100, 2)
    unique_score += perc_check_unique
    print(f'The percentage	of	uniqueness	for	{df_name} :	{perc_check_unique}')
uniqueness	= unique_score/len(list(df_set))
print(f'유일성	품질지수:	{round(uniqueness, 2)}%')

The percentage	of	uniqueness	for	df_0 :	40.6
The percentage	of	uniqueness	for	df_1 :	100.0
The percentage	of	uniqueness	for	df_2 :	100.0
The percentage	of	uniqueness	for	df_3 :	99.98
The percentage	of	uniqueness	for	df_4 :	99.95
The percentage	of	uniqueness	for	df_5 :	99.96
The percentage	of	uniqueness	for	df_6 :	99.97
유일성	품질지수:	91.49%
